In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
import keras

Using TensorFlow backend.


In [2]:
tfds.disable_progress_bar()
(train_ds, test_ds), ds_info = tfds.load(
    'food101', 
    as_supervised=True,
    with_info=True,
    split=['train[:90%]', 'train[90%:]'],
)

some_samples = [x[0] for x in iter(train_ds.take(32))]

fig = plt.figure(figsize=(16, 8))
for j in range(len(some_samples)):
    ax = fig.add_subplot(4, 8, j+1)
    ax.imshow(some_samples[j])
    plt.xticks([]), plt.yticks([])
plt.show()

In [3]:
INP_SIZE = 160
NUM_EPOCHS = 10
BATCH_SIZE = 32
num_classes = 101


def prepare(img, label):
    img = tf.cast(img, tf.float32)/256.
    print ()
    return tf.image.resize(img, (INP_SIZE, INP_SIZE)), tf.one_hot(label, num_classes, dtype=tf.float32)

train_ds = train_ds.shuffle(buffer_size=1000)
train_ds = train_ds.map(prepare)
train_ds = train_ds.batch(BATCH_SIZE, drop_remainder=True)

test_ds = test_ds.shuffle(buffer_size=1000)
test_ds = test_ds.map(prepare)
test_ds = test_ds.batch(BATCH_SIZE, drop_remainder=True)

In [4]:
if 0: # Обучение модели с нуля

    EXP_NAME = 'scratch'
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(INP_SIZE, INP_SIZE, 3),
        include_top=False,
        weights='imagenet',
    )
    base_model.trainable = True

else: # Transfer Learning

    EXP_NAME = 'transfer'
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(INP_SIZE, INP_SIZE, 3),
        include_top=False,
        weights='imagenet',
    )
    base_model.trainable = True # Fine-tuning весов предобученной модели
    # base_model.trainable = False # Заморозка весов предобученной модели

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(num_classes, activation='softmax'),
])

In [5]:
LEARNING_RATE = 0.0001
optimizer = tf.keras.optimizers.RMSprop(lr=LEARNING_RATE)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/'+EXP_NAME, 
    write_graph=False, update_freq=100, profile_batch=0)

In [6]:
%%time

history = model.fit(
    train_ds,
    epochs=NUM_EPOCHS,
    validation_data=test_ds,
    callbacks=[tensorboard_callback])

Epoch 1/10
2130/2130 [==============================] - 228s 107ms/step - loss: 2.4543 - accuracy: 0.4108 - val_loss: 1.8981 - val_accuracy: 0.5273
Epoch 2/10
2130/2130 [==============================] - 229s 108ms/step - loss: 1.4499 - accuracy: 0.6208 - val_loss: 1.5114 - val_accuracy: 0.6245
Epoch 3/10
2130/2130 [==============================] - 229s 107ms/step - loss: 1.1120 - accuracy: 0.7020 - val_loss: 1.7111 - val_accuracy: 0.6214
Epoch 4/10
2130/2130 [==============================] - 228s 107ms/step - loss: 0.8715 - accuracy: 0.7576 - val_loss: 1.6448 - val_accuracy: 0.6360
Epoch 5/10
2130/2130 [==============================] - 418s 196ms/step - loss: 0.6850 - accuracy: 0.8047 - val_loss: 2.0219 - val_accuracy: 0.6250
Epoch 6/10
2130/2130 [==============================] - 514s 241ms/step - loss: 0.5457 - accuracy: 0.8386 - val_loss: 1.9210 - val_accuracy: 0.6198
Epoch 7/10
2130/2130 [==============================] - 525s 246ms/step - loss: 0.4347 - accuracy: 0.8680 - val_

In [7]:
%%time

model.evaluate(test_ds)

236/236 [==============================] - 7s 30ms/step - loss: 2.7477 - accuracy: 0.6213
Wall time: 7.58 s


[2.7476658821105957, 0.6212923526763916]

In [8]:
%load_ext tensorboard
%tensorboard --logdir logs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2020-05-27 22:17:59.419715: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_101.dll
Traceback (most recent call last):
  File "d:\anaconda\anaconda3\envs\cnn\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "d:\anaconda\anaconda3\envs\cnn\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\anaconda\anaconda3\envs\CNN\Scripts\tensorboard.exe\__main__.py", line 7, in <module>
  File "d:\anaconda\anaconda3\envs\cnn\lib\site-packages\tensorboard\main.py", line 75, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "d:\anaconda\anaconda3\envs\cnn\lib\site-packages\absl\app.py", line 299, in run
    _run_main(main, args)
  File "d:\anaconda\anaconda3\envs\cnn\lib\site-packages\absl\app.py", line 250, in _run_main
    sys.exit(main(argv))
  File "d:\anaconda\anaconda3\envs